In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
from skimage import io
import json
import pickle

%matplotlib inline

In [ ]:
# define class for JSON object
class myEncoder(json.JSONEncoder):
    def default(self, obj):
        return {'_python_object': pickle.dumps(obj)}

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(str(dct['_python_object']))
    return dct

In [ ]:
# load metadata_table
df = pd.read_csv('C:\\Users\\Lina\\Anaconda3\\notebook\\data\\metadata\\metadata_table_all.csv')
structure = "TTF-1_cells"
species = "mus musculus"

with open('./output/green_cells/data_greencells.json', 'wb') as f:
    json.dump([], f)
    
for subdir, dirs, files in os.walk('G:/2016/Summer/LungMAP/Image_data'):
    for file in files:
        filepath = subdir + os.sep + file
        
        if "LMH" not in filepath and "NKX2.1" in filepath and "meta.json" not in filepath:
            im = cv2.imread(filepath)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            origin = im.copy()
            age = df[df["img_file"]==file]["age_label"].any()
            
            if (df[df["img_file"]==file][df[df["img_file"]==file]["probe_label"] == "TTF-1"]["color"].any() == "Green"):
                R, G, B = cv2.split(im)
                G_norm = cv2.normalize(G,G,0, 175, cv2.NORM_MINMAX)
                im = cv2.merge((R, G_norm, B))
                thresh = cv2.inRange(im, np.array([0,55,40]), np.array([60,255,255]))
            elif (df[df["img_file"]==file][df[df["img_file"]==file]["probe_label"] == "TTF-1"]["color"].any() == "Red"):
                thresh = cv2.inRange(im, np.array([68,0,50]), np.array([255,60,255]))
                
            # find contours
            c=0
            if (df[df["img_file"]==file]["magnification"].any() == "60X"):
                minarea = 200
            elif (df[df["img_file"]==file]["magnification"].any() == "20X"):
                minarea = 20
            finalcontours = []
            im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
            for i in range(0, len(contours)):
                a = cv2.contourArea(contours[i])
                if (a>minarea):
                    finalcontours.append(contours[i])
                    c=c+1

            # draw contours       
            #for ia in range(0, len(finalcontours)):
                #temp2 = cv2.drawContours(origin,finalcontours,ia,(250,220,55),thickness=2, lineType=2,maxLevel=0)
            #plt.figure(figsize=(38, 8))
            #plt.title(file)
            #plt.imshow(temp2)

            center = []
            area = []
            contours_out = []
            for ct in finalcontours:
                M = cv2.moments(ct)
                ctX = int(M["m10"] / M["m00"])
                ctY = int(M["m01"] / M["m00"])
                cv2.circle(im, (ctX, ctY), 7, (255, 255, 255), -1)
                a = cv2.contourArea(ct)
                center.append((ctX,ctY))
                area.append(a)
                #contours_out.append(ct)

            with open('./output/green_cells/data_greencells.json', 'rb') as feedsjson:
                feeds = json.load(feedsjson)                        
            with open('./output/green_cells/data_greencells.json', 'wb') as outfile:
                output = { 'area': area,
                      'center': center,
                      #'contour_(x,y)': contours_out,
                      'img_file': file,
                      'age': age,
                      'species': species,
                      'structure': structure,
                      'number of cells': len(finalcontours),
                      'total area of cells': sum(area)
                    }
                
                feeds.append(output)
                json.dump(feeds, outfile)
                #json.dump(feeds, outfile, cls = myEncoder)

                